# `Step 10: Predict`

# 0- `Config`

In [ ]:
import pandas as pd 
import numpy as np 
import pickle 

In [ ]:
try: 
    if manager == 1:
        print("s1_load.ipynb running from MANAGER")
except: 
    %run s0_config.ipynb 
 

# 1-) `Load`
- 0-) Default
- 1-) SVM
- 2-) SGD
- 3-) Decision Tree
- 4-) Random Forest
- 5- Gradient Boosting

## 0-) `Default`

In [ ]:
# Basic 
import_name = "subject_outlier"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_prediction}", f"{import_name}" + ".pkl"))  , 'rb') as file: 
    subject_outlier = pickle.load(file)   

import_name = "subject_autoencoder"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_prediction}", f"{import_name}" + ".pkl"))  , 'rb') as file: 
    subject_autoencoder = pickle.load(file)   

import_name = "subject_dimension" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_prediction}", f"{import_name}" + ".pkl"))  , 'rb') as file: 
    subject_dimension = pickle.load(file)   

## 0-) `NN`

In [ ]:
# Subject
import_name = "NN_All_Models"
with open(os.path.join(access_data_path(f"{process_barcode}/model", f"{import_name}" + ".pkl"))  , 'rb') as file: 
    NN_All_Models = pickle.load(file)   

# Subject
import_name = "NN_All_Encoders"
with open(os.path.join(access_data_path(f"{process_barcode}/model", f"{import_name}" + ".pkl"))  , 'rb') as file: 
    NN_All_Encoders = pickle.load(file)   


## 1-) `SVM`

In [ ]:
# Subject
import_name = "SVM_All_Models"
with open(os.path.join(access_data_path(f"{process_barcode}/model", f"{import_name}" + ".pkl"))  , 'rb') as file: 
    SVM_All_Models = pickle.load(file)   


## 2-) `SGD`

In [ ]:
# Subject
import_name = "SGD_All_Models"
with open(os.path.join(access_data_path(f"{process_barcode}/model", f"{import_name}" + ".pkl"))  , 'rb') as file: 
    SGD_All_Models = pickle.load(file)   


## 3-) `Decision Tree`

In [ ]:
# Subject
import_name = "DT_All_Models"
with open(os.path.join(access_data_path(f"{process_barcode}/model", f"{import_name}" + ".pkl"))  , 'rb') as file: 
    DT_All_Models = pickle.load(file)   


## 4-) `Random Forest`

In [ ]:
# Subject
import_name = "RF_All_Models"
with open(os.path.join(access_data_path(f"{process_barcode}/model", f"{import_name}" + ".pkl"))  , 'rb') as file: 
    RF_All_Models = pickle.load(file)   


---
# 4.A- `Prediction`
- 0-) Config
- 1-) Text to Predict
- 2-) Preprocess Text
- 3-) Predict

## 0- `Config`

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
import torch

In [ ]:
# Example text format for prediction
gene_marker = "JCHAIN"  # 'Unknown' acts as a placeholder for Cell_Type 
gene_expression = 6.496516  # Example expression level 

## 1- `Key to Predict`

In [ ]:
# Initialize the model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token 

model     = GPT2Model.from_pretrained('gpt2')

text_to_predict = "Unknown " + gene_marker
inputs = tokenizer(text_to_predict, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Generate text embedding
with torch.no_grad():
    outputs = model(**inputs)
    text_embedding = outputs.last_hidden_state.mean(dim=1).numpy()  # Convert to numpy array for compatibility



gene_expression_array = np.array([[gene_expression]])  # Shape (1, 1) to match embedding shape for concatenation

# Combine gene expression and text embeddings
combined_input = np.hstack([gene_expression_array, text_embedding])
# Split combined_input back into gene_expression_array and text_embedding for prediction
gene_expression_array = combined_input[:, :1]  # Assuming gene expression is the first column
text_embedding = combined_input[:, 1:]  # The rest are the text embedding columns


In [ ]:
index_NN       = 0  
NN_All_Model   = NN_All_Models[index_NN] 
NN_All_Encoder = NN_All_Encoders[index_NN]  

In [ ]:
NN_All_Models

In [ ]:

# Make predictions using the split inputs
prediction = NN_All_Model.predict([gene_expression_array, text_embedding])

# Proceed with the rest of your prediction process
predicted_index = np.argmax(prediction, axis=1)
predicted_label = NN_All_Encoder.inverse_transform(predicted_index)
print("Predicted Cell Type Label:", predicted_label[0]) 

---
# 4-B `Prediction`
- 0-) Config
- 1-) Text to Predict
- 2-) Preprocess Text
- 3-) Predict

## 0- `Config`

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
import torch

## 1- `Key to Predict`

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
# Assuming 'gene_marker' is the gene marker and 'gene_expression' is the expression level for prediction
gene_marker = "JCHAIN"
gene_expression = 6.496516	 # Example expression level 

## 2- `Preprocess Text`

In [ ]:
# Generate text embeddings
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token 

model = GPT2Model.from_pretrained('gpt2')
inputs = tokenizer("Unknown " + gene_marker, return_tensors="pt", padding=True, truncation=True, max_length=512)
with torch.no_grad():
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).numpy()

# Combine gene expression and text embeddings
combined_input = np.hstack([np.array([[gene_expression]]), embedding])


## 3- `Prediction`

In [ ]:
# Prediction using the SVM model
index_SVM = 0  # Assuming you have a list of SVM models and you want to use the first one
SVM_All_Model = SVM_All_Models[index_SVM]  # Assuming SVM_All_Models is a list of trained SVM models
prediction = SVM_All_Model.predict(combined_input)

print("Predicted Cell Type:", prediction[0])


In [ ]:
index_SVM = 0  
SGD_All_Model = SGD_All_Models[index_SVM] 
prediction = SGD_All_Model.predict(combined_input)

print("Predicted Cell Type:", prediction[0])


In [ ]:
index_SVM = 0  
DT_All_Model = DT_All_Models[index_SVM] 
prediction   = DT_All_Models.predict(combined_input)

print("Predicted Cell Type:", prediction[0])


In [ ]:
index_SVM = 0  
RF_All_Model = RF_All_Models[index_SVM] 
prediction   = RF_All_Model.predict(combined_input)

print("Predicted Cell Type:", prediction[0])


In [ ]:
try:
    index_SVM = 0  
    GB_All_Model = GB_All_Models[index_SVM] 
    prediction = GB_All_Model.predict(combined_input)

    print("Predicted Cell Type:", prediction[0])
except:
    pass 